In [ ]:
def print_results(tuner):
    print("Final prompt:", tuner.final_prompt)
    print("Start prompt metric: ", tuner.init_metric)
    print("Final prompt metric: ", tuner.final_metric)

# PromptTuner

Точка входа - ассистент

Он позволяет оптимизировать имеющийся промпт под определенную задачу

Задачу можно задавать датасетом, подаваемым в функцию run, но некоторые оптимизаторы могут работать и без него

In [ ]:
from coolprompt.assistant import PromptTuner
tuner = PromptTuner()

In [ ]:
# Запуск без датасета
tuner.run(start_prompt="Write an essay about autumn")

Ассистент в режиме оптимизации под датасет может решать одну из двух задач: классификацию или генерацию

In [ ]:
from datasets import load_dataset

# Классификация

sst2 = load_dataset("stanfordnlp/sst2")
class_dataset = sst2['train']['sentence'][:100]
class_targets = sst2['train']['label'][:100]

tuner.run(
    start_prompt="Perform Sentiment Classification task.",
    task="classification",
    dataset=class_dataset,
    target=class_targets,
    metric="accuracy"
)

print_results(tuner)

In [ ]:
# Генерация

samsum = load_dataset("knkarthick/samsum")
gen_dataset = samsum['train']['dialogue'][:100]
gen_targets = samsum['train']['summary'][:100]

tuner.run(
    start_prompt="Summarize the text",
    task="generation",
    dataset=gen_dataset,
    target=gen_targets,
    metric="meteor"
)

print_results(tuner)

## Language model

При инициализации юзер может подать уже имеющуюся LLM-ку, 

или инициализировать асситента без аргументов, и будет использована дефолтная модель

In [ ]:
from langchain_community.llms import VLLM

my_model = VLLM(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    trust_remote_code=True,
    dtype='float16',
)

tuner_with_custom_llm = PromptTuner(model=my_model)
tuner_with_custom_llm.run(start_prompt="Write an essay about autumn")

Также можно изменять конфиг дефолтной модели

In [ ]:
from coolprompt.language_model.llm import DefaultLLM

changed_model = DefaultLLM.init(langchain_config={
    'max_new_tokens': 1000,
    "temperature": 0.0,
})

tuner_with_changed_llm = PromptTuner(model=changed_model)
tuner_with_changed_llm.run(start_prompt="Write an essay about autumn")

## Оптимизатор

Фреймворк поддерживает несколько различных алгоритмов оптимизации промптов

- HyPE
- DistillPrompt
- ReflectivePrompt

Выбор алгоритма производится в параметрах метода run

### HyPE
Мы уже видели его работу до этого - используя заранее заданную системную инструкцию просит модель оптимизировать поданный промпт. оптимизация происходит в одну итерацию

In [ ]:
tuner.run(
    start_prompt="Perform Sentiment Classification task.",
    task="classification",
    method="hype",
    dataset=class_dataset,
    target=class_targets,
    metric="accuracy"
)

print_results(tuner)

### DistillPrompt

Метод автопромптинга, основанный на последовательной дистилляции и агрегации знаний LLM

In [ ]:
tuner.run(
    start_prompt="Perform Sentiment Classification task.",
    task='classification',
    dataset=class_dataset,
    target=class_targets,
    method='distill',
    use_cache=True,
    num_epochs=1
)

print_results(tuner)

### ReflectivePrompt

Метод автопромптинга на основе эволюционных алгоритмов, использующий подход рефлексивной эволюции для более точного и расширенного поиска оптимальных промптов

In [ ]:
tuner.run(
    start_prompt="Perform Sentiment Classification task.",
    task='classification',
    dataset=class_dataset,
    target=class_targets,
    method='reflective',
    problem_description='sentiment classification',
    use_cache=False,
    population_size=4,
    num_epochs=3
)

print_results(tuner)

## Evaluator

Если вы укажете датасет для оптимизации, фреймворк позволяет измерять метрики, получаемые при использовании определенного промпта

На данный момент поддерживаются
- accuracy и f1 для классификации
- meteor, rouge и bleu для генерации

Метрики находятся в публичных полях ассистента

In [ ]:
print("Start prompt metric: ", tuner.init_metric)
print("Final prompt metric: ", tuner.final_metric)